In [1]:
%matplotlib inline

import mido
import numpy as np
from math import ceil

In [53]:
# Formulas to convert time to beat

#500000us per beat
#song_length = simp.length
#bpm = 60/(500000/1e6) BPM

In [2]:
def import_mid(midfile):
    return mido.MidiFile('data/' + midfile)    

In [3]:
# I want type 0 or 1

# type 0 (single track): all messages are saved in one track
# type 1 (synchronous): all tracks start at the same time
# type 2 (asynchronous): each track is independent of the others

In [4]:
twin = mido.MidiFile('data/twinkle.mid')
twin2 = mido.MidiFile('data/twinkle_twinkle.mid')
simp = mido.MidiFile('data/simple_test.mid')
simp2 = mido.MidiFile('data/simple_test_2notes.mid')
song = mido.MidiFile('data/bags_groove_jh.mid')
mond = mido.MidiFile('data/mond_3.mid')
abba = mido.MidiFile('data/ABBA_-_Dancing_Queen.mid')
bach = mido.MidiFile('data/bach_variations_sn.mid')
cop = mido.MidiFile('data/appspg13.mid')

In [52]:
# creates notelist - a list for all the notes

midifile = twin

# note_list = (note, start, duration)
# have to address if notes are not perfectly inside a tick_step

resolution = 8 # eighth notes

# this gives # of ticks in a column of the piano sheet
tick_step = round(float(midifile.ticks_per_beat/resolution))

note_range = (20, 90)
active_notes = [0] * abs(np.diff(note_range)[0])

counter = 0
notelist = []

rawmsgs = [msg for track in midifile.tracks for msg in track]

# have to reset counter when a new track is detected
for msg in rawmsgs:
    if msg.type not in set(['note_on','note_off']):
        continue
    
    #counter += ceil(msg.time/tick_step)
    counter += msg.time
    
    if msg.type == 'note_on' and msg.velocity > 0:        
        active_notes[msg.note - note_range[0]] = counter
        
    elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
        # fill everything up to this with 1s
        start = active_notes[msg.note - note_range[0]]  #round this section
               
        notelist.append((msg.note, int(ceil(start/tick_step)), int(ceil((counter-start)/tick_step))))
        active_notes[msg.note - note_range[0]] = 0

notelist = sorted(notelist, key=lambda x: x[1])

In [25]:
steps = int(ceil(counter/tick_step))
music_mat = np.zeros([steps, int(np.diff(note_range))])

In [50]:
for nt in notelist:
    r = range(nt[1], nt[2])
    c = [nt[0] - note_range[0]] * len(r)
    music_mat[r,c] = np.array([1] * (len(r)-1) + [2])


In [140]:
def get_bpm(x):
    return 60/float(x) * 1e6

In [142]:
f = mond.tracks[0][20]

In [143]:
f.type

tmp = []
for x in mond.tracks[0]:
    if x.type == 'set_tempo':
        tmp.append(x.tempo)

In [144]:
[get_bpm(x) for x in tmp]

[166.7398469328205,
 159.9799491797028,
 166.7398469328205,
 159.44979191802153,
 166.7398469328205,
 160.49002955691378,
 166.7398469328205,
 164.1398252457994,
 166.7398469328205,
 143.30000812033379,
 164.66010038777455,
 160.49002955691378,
 160.49002955691378,
 161.00985380305275,
 161.53001227628093,
 162.0601080940921,
 162.58003679728168,
 163.1002087682672,
 163.6201503669182,
 164.1398252457994,
 157.3700320248015,
 162.0601080940921,
 162.58003679728168,
 163.1002087682672,
 163.6201503669182,
 164.1398252457994,
 164.1398252457994,
 163.6201503669182,
 163.1002087682672,
 162.58003679728168,
 162.0601080940921,
 161.53001227628093,
 161.00985380305275,
 160.49002955691378,
 159.9799491797028,
 159.44979191802153,
 158.92987219389445,
 158.40998616552787,
 157.8801952451748,
 157.3700320248015,
 156.8500330692153,
 121.92990656104827,
 83.37003930897355,
 166.7398469328205,
 162.0601080940921,
 166.7398469328205,
 163.6201503669182,
 166.7398469328205,
 166.7398469328205,
 1

In [ ]:
# sum up the ticks on each channel to see what they come out to

In [55]:
mond.ticks_per_beat

480

In [150]:
q = mond.tracks[2][10]

In [151]:
q.time

120

### Find the total # of beats for all tracks in mid file

In [167]:
# Finds the total # of beats for all tracks in the mid file

total = []
for track in cop.tracks:
    sub = 0
    for x in track:
        if x.type in ('note_on','note_off'):
            sub += x.time
    total.append(sub)
print total

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1136841, 1103742, 1057048, 1016096, 1114374, 1115442, 1118249, 1096334, 1090780]


### Find the lowest resolution

In [53]:
lowest_res_num = 4 # Lowest resolution.  4 - qtr note.
upper_bound = 102
lower_bound = 24
span = upper_bound - lower_bound

tpb = mond.ticks_per_beat
lowest_res = tpb / lowest_res_num

In [40]:
# clocks_per_click = parts per quarter. 24 clicks per quarter note
# tempo - 600000 micro_secs/quarter note = 100bpm

2343